## Importing libraries

From the *Ordinal Deep Learning* package, we import the methods that will allow us to work with ordinal datasets.

We also import methods from libraries such as *pytorch* and *torchvision* that will allow us to process and work with the datasets.


In [1]:
from dlordinal.datasets import FGNet, Adience
from torchvision.transforms import ToTensor, Compose
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

## FGNet

To make use of the [FGNet dataset](https://yanweifu.github.io/FG_NET_data/), an instance of it will be created where the following fields will be specified:

* __root__: an attribute that defines the path where the dataset will be downloaded and extracted.
* __download__: an attribute that indicates the desire to perform the dataset download.
* __process_data__: an attribute that allows indicating to the method whether the data should be preprocessed for working with it, in case the user does not want to perform their own preprocessing.

In [2]:
fgnet = FGNet(root='./datasets/fgnet', download=True, process_data=True)

Files already downloaded and verified
Files already processed and verified
Files already split and verified


Once the data has been downloaded, extracted, and preprocessed, we can load it to subsequently make use of it for training and validating a model.

After decompressing the dataset and processing it, we will see that a folder named *FGNET* is created, and inside it, we will find the *train* and *test* folders.

In [3]:
train_data = ImageFolder(
    root="./datasets/fgnet/FGNET/train", transform=Compose([ToTensor()])
)
test_data = ImageFolder(
    root="./datasets/fgnet/FGNET/test", transform=Compose([ToTensor()])
)

As an additional data processing step, we are going to show how we can obtain the number of classes in the dataset and how we can create a partition for validation.

In [4]:
# Obtain the number of classes
num_classes = len(train_data.classes)

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(sss.split(X=np.zeros(len(train_data)), y=train_data.targets))
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(train_data, train_idx)
val_data = Subset(train_data, val_idx)

### Adience

The [Adience dataset](https://talhassner.github.io/home/projects/Adience/Adience-data.html) does not allow direct download like FGNet, so a series of instructions must be followed to be able to download it.

* Create a directory `datasets` where all your datasets are going to be saved.
* Create a directory `datasets/adience` where the Adience dataset is going to be stored.
* Create a directory `datasets/adience/folds` where the different fold files are going to be downloaded.
* Download files fold_0_data.txt - fold_4_data.txt and place them in `datasets/adience/folds`.
* Download aligned.tar.gz and place it in `datasets/adience`.

When all the files are downloaded, an Adience object can be created passing the following parameters:
* __root__: path of the root directory where all the datasets are stored. In this case, specify the `datasets` directory.
* __train__: determines whether the train split if going to be used. If False, the test partition is used.
* __test_size__: indicate the proportion of samples used for the test set.



In [5]:
adience = Adience(
    root="./datasets/",
    train=True,
    test_size=0.2,
)

Now, the `adience` object can be used as any other `VisionDataset` from `torchvision`.

In [6]:
print(f'Number of samples in the Adience dataset: {len(adience)}')
print(f'Targets of the adience dataset: {adience.targets}')
print(f'Classes of the adience dataset: {adience.classes}')
print(f'3rd sample of the adience dataset: {adience[3]}')

Number of samples in the Adience dataset: 14012
Targets of the adience dataset: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0